# Ficha de trabalho 4

#### Ex.1) Considera o dataset cachexia. Lê os dados usando a função read_csv do pandas (dados e metadados).

In [1]:
from pandas import read_csv
data_metab = read_csv("data_cachexia.csv", sep = ",", index_col =0)
meta_metab = read_csv("meta_cachexia.csv", sep = ",", index_col=0)
print(data_metab.shape)
print(meta_metab.shape)

(77, 63)
(77, 1)


#### Ex.2) Aplica a transformação necessária para que os dados seja escalonados para ter média 0 e desvio padrão 1. Verifica que as médias de todas as colunas é aproximadamente zero.

In [2]:
from sklearn import preprocessing

input_data = data_metab.values
output_data = meta_metab.values[:, 0]

input_data_scale = preprocessing.scale(input_data)

print("Média global:", input_data_scale.mean())
print("Desvio padrão global:", input_data_scale.std())

Média global: -1.8675365658506033e-17
Desvio padrão global: 1.0


#### Ex.3) Cria dois conjuntos de dados treino e teste para serem usados na criação e validação de modelos de aprendizagem máquina. Considera 30% das amostras para formar o conjunto de teste.

In [3]:
import numpy as np

numtst = int(input_data_scale.data.shape[0] * 0.3)
print("Numero de exemplos para teste: ", numtst)

indices = np.random.permutation(len(input_data))

train_in = input_data_scale[indices[:-numtst]]
train_out = output_data[indices[:-numtst]]

test_in  = input_data_scale[indices[-numtst:]]
test_out = output_data[indices[-numtst:]]

train_in.shape, train_out.shape, test_in.shape, test_out.shape

Numero de exemplos para teste:  23


((54, 63), (54,), (23, 63), (23,))

#### Ex.4) Constroi diversos tipos de modelos de classificação treinando e testando com os conjuntos criados anteriormente. Calcula, para cada um dos modelos no test set, as métricas de erro PECC e F1-score, e calcula a matriz de confusão. Considere como classe positiva a classe "cachexic"

##### Modelos

In [12]:
# score -> mean accuracy on the given test data and labels (the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true)

In [14]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
knn_model.fit(train_in, train_out)
prev_knn = knn_model.predict(test_in)
print(knn_model.score(test_in, test_out))

0.6086956521739131


In [8]:
from sklearn import tree

tree_model = tree.DecisionTreeClassifier()
tree_model = tree_model.fit(train_in, train_out)
prev_tree = tree_model.predict(test_in)

print(tree_model.score(test_in, test_out))

0.6086956521739131


In [11]:
from sklearn.naive_bayes import GaussianNB

gnb_model = GaussianNB()
gnb_model = gnb_model.fit(train_in, train_out)
prev_gnb = gnb_model.predict(test_in)

print(gnb_model.score(test_in, test_out))

0.6086956521739131


##### PECC Score

In [15]:
def pecc_func (prev, real):
    return sum([ (prev[i]==real[i]) for i in range(len(prev)) ])/ len(prev)

In [16]:
print ("PECC: ", pecc_func(prev_knn, test_out))

PECC:  0.6086956521739131


In [17]:
print ("PECC: ", pecc_func(prev_tree, test_out))

PECC:  0.6086956521739131


In [18]:
print ("PECC: ", pecc_func(prev_gnb, test_out))

PECC:  0.6086956521739131


##### Confusion Matrix

In [37]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_out, prev_knn)
# rows -> true values (test_out)
# cols -> predicted values (prev_knn)

array([[11,  8],
       [ 1,  3]], dtype=int64)

In [20]:
confusion_matrix(test_out, prev_tree)

array([[11,  8],
       [ 1,  3]], dtype=int64)

In [21]:
confusion_matrix(test_out, prev_gnb)

array([[11,  8],
       [ 1,  3]], dtype=int64)

##### F1 Score

In [23]:
from sklearn.metrics import f1_score

f1_score(test_out, prev_knn, average='macro')

0.5548387096774193

In [24]:
f1_score(test_out, prev_tree, average='macro')

0.5548387096774193

In [25]:
f1_score(test_out, prev_gnb, average='macro')

0.5548387096774193

In [27]:
# All the evaluation metrics for the different models, KNN Classifier, Decision Tree and Gaussion, have identical values. The models show a fair aptitude (scores of 0,5 and 0,6) to predict the correct value. Additionally, we can also infer that the model is more accurate at predicting true negatives.

#### Ex.5) Usando apenas o melhor modelo do exercício anterior, compare o seu desempenho considerando dados com transformação logaritmica e posteriormente standardizados

In [42]:
log_data = np.log2(data_metab)

In [45]:
log_input_data = log_data.values
output_data = meta_metab.values[:, 0]

log_input_data_scale = preprocessing.scale(log_input_data)

print("Média global:", log_input_data_scale.mean())
print("Desvio padrão global:", log_input_data_scale.std())

Média global: 5.0899526010438016e-17
Desvio padrão global: 1.0


In [46]:
import numpy as np

numtst = int(log_input_data_scale.data.shape[0] * 0.3)
print("Numero de exemplos para teste: ", numtst)

indices = np.random.permutation(len(log_input_data))

log_train_in = log_input_data_scale[indices[:-numtst]]
log_train_out = log_input_data_scale[indices[:-numtst]]

log_test_in  = log_input_data_scale[indices[-numtst:]]
log_test_out = log_input_data_scale[indices[-numtst:]]

log_train_in.shape, log_train_out.shape, log_test_in.shape, log_test_out.shape

Numero de exemplos para teste:  23


((54, 63), (54, 63), (23, 63), (23, 63))

In [ ]:
# As the variables are continuous, the models need to be regression models.

In [49]:
reg_tree_model = tree.DecisionTreeRegressor()
reg_tree_model = reg_tree_model.fit(log_train_in, log_train_out)
prev_tree = reg_tree_model.predict(log_test_in)

print(reg_tree_model.score(log_test_in, log_test_out))

0.11641166794360254


In [51]:
# The best possible outcome is 1.0 and as such this is a low score. Therefore, the model does not have a good capability to accurately predict the values.